In [13]:
#Joseph K Blankenship, Pramiti Barua, Jaswitha Ravala
#CS 5050 Project: Sentiment Classification of Amazon Reviews with Hybrid Naive Bayes Approach

In [14]:

import pandas as pd
import re
import nltk
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import sentiwordnet as swn
from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import CountVectorizer


In [15]:
nltk.download('wordnet')
ar = pd.read_csv("Amazon_Consumer_Reviews_of_Amazon_Products.csv")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JoeKa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
#nltk.download('sentiwordnet')

In [17]:
#nltk.download('averaged_perceptron_tagger')

In [18]:
#nltk.download('punkt')

In [19]:
stop = stopwords.words('english')

In [20]:
new_stop = ["i","you","he","she","it","we","they","them","us","him", "me", "her"]
stop_no_pro = []
for word in stop:
    if word in new_stop:
        continue 
    else:
        stop_no_pro.append(word)

In [21]:
#stop_no_pro = nltk.tokenize(stop_no_pro)

In [22]:
review_list_reg = []
for review in ar['reviews.text']:
     review_list_reg.append(review)

In [24]:
#This is function to generate the normalized senti-score for our reviews.

def senti_calc(word, tag):
    pos_score = 0
    neg_score = 0
    
    if 'NN' in tag and len(list(swn.senti_synsets(word, 'n')))>0:
        syn_set = list(swn.senti_synsets(word, 'n'))
    elif 'VB' in tag and len(list(swn.senti_synsets(word, 'v')))>0:
        syn_set = list(swn.senti_synsets(word, 'v'))
    elif 'JJ' in tag and len(list(swn.senti_synsets(word, 'a')))>0:
        syn_set = list(swn.senti_synsets(word, 'a'))
    elif 'RB' in tag and len(list(swn.senti_synsets(word, 'r')))>0:
        syn_set = list(swn.senti_synsets(word, 'r'))
    elif 'PRP' in tag:
        neg_score += 0.3
        #pos_score += 0.3
    else:
        return (0,0)
    
    if ('PRP' in tag):
        return (pos_score, neg_score)
    else:
        for syn in syn_set:
            pos_score += syn.pos_score()
            neg_score += syn.neg_score()
        return (pos_score/len(syn_set), neg_score/len(syn_set))



In [25]:
pos_senti_score = []
neg_senti_score = []
for review in review_list_reg:
    #no_punct = tokenizer.tokenize(review)
    tokens = nltk.word_tokenize(review)
    tags = nltk.pos_tag(tokens)
    length = len(tags)
    pos_score = 0
    neg_score = 0
    for word,tag in tags:
        pos_score += senti_calc(word,tag)[0]
        neg_score += senti_calc(word,tag)[1]
    pos_senti_score.append(pos_score/length)
    neg_senti_score.append(neg_score/length)


In [26]:
review_rating = []
for rating in ar['reviews.rating']:
     review_rating.append(rating)

In [27]:
classifier = MultinomialNB()

In [28]:
vectorizer = CountVectorizer(stop_words = stop_no_pro)
X = vectorizer.fit_transform(review_list_reg)

C:\Users\JoeKa\anaconda3\envs\cs5050\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['it', 'she', 'you'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [29]:
matrix = X.toarray()

In [47]:
matrix.shape

(5000, 5285)

In [30]:
length = len(pos_senti_score)
for x in range(length):
    pos_senti_score[x] = int(pos_senti_score[x] * 100)

In [31]:
length = len(neg_senti_score)
for x in range(length):
    neg_senti_score[x] = int(neg_senti_score[x] * 100)

In [32]:
matrix2 = np.c_[matrix,neg_senti_score]

In [33]:
matrix_f = np.c_[matrix2,pos_senti_score]

In [34]:
X_train, X_test, y_train, y_test = train_test_split(matrix_f, review_rating)

In [35]:
classifier.fit(X_train, y_train)

MultinomialNB()

In [36]:
y_pred = classifier.predict(X_test)

In [37]:
accuracy = accuracy_score(y_test,y_pred)

In [38]:
accuracy

0.708

In [39]:
report = classification_report(y_test,y_pred)

In [40]:
print(report)

              precision    recall  f1-score   support

           1       1.00      0.17      0.29        12
           2       1.00      0.15      0.27        13
           3       0.25      0.02      0.03        61
           4       0.46      0.27      0.34       312
           5       0.75      0.94      0.83       852

    accuracy                           0.71      1250
   macro avg       0.69      0.31      0.35      1250
weighted avg       0.66      0.71      0.66      1250

